In [1]:
#pip install pandas statsmodels numpy scipy linearmodels

# Replication of Card and Krueger (1994) DiD Analysis in Python

This notebook replicates the Stata analysis performed on the Card and Krueger dataset regarding the impact of a minimum wage increase in New Jersey on fast-food employment, using Pennsylvania as a control group. We will perform Difference-in-Differences (DiD) estimation using various methods.

**Dataset:** `Card_Krueger_1994.csv` 
**Variables:**
*   `sheet`: Unique store identifier
*   `fte`: Full-time equivalent employment
*   `nj`: Dummy variable, 1 if the restaurant is in New Jersey, 0 if in Pennsylvania
*   `after`: Dummy variable, 1 for the period after the minimum wage increase, 0 for before
*   `njafter`: Interaction term (`nj` * `after`), 1 for NJ restaurants after the increase
*   `dfte`: Change in `fte` between the 'before' and 'after' periods (`fte_after - fte_before`) for each restaurant.

### Setup: Import Libraries and Load Data

First, we need to import the necessary Python libraries and load the dataset (`Did.csv`) into a pandas DataFrame. We'll also display the first few rows and some basic information about the data to ensure it's loaded correctly.

In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats
from linearmodels.panel import PanelOLS
import os

# --- Configuration ---
# Define the file path 
# Make sure 'Card_Krueger_1994.csv' is in your current working directory or provide the full path
file_path = "Card_Krueger_1994.csv" 
# Set display precision for floating-point numbers in pandas output
pd.options.display.float_format = '{:.3f}'.format 

# --- Load Data ---
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded data from: {file_path}")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure the CSV file is in the correct directory or update the 'file_path'.")
    # Exit or raise an error if the file isn't found, as subsequent steps depend on it.
    # For this example, we'll proceed assuming the file loads, but in practice, handle this robustly.
    df = None # Set df to None if loading fails

# --- Initial Data Exploration (only if df is loaded) ---
if df is not None:
    print("\nFirst 5 rows of the dataframe:")
    # Drop the unnamed index column if it exists from CSV saving/loading
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    print(df.head())
    
    print("\nDataFrame Info:")
    df.info()
    
    print("\nDescriptive Statistics:")
    print(df.describe())

    # Check for missing values
    print("\nMissing values per column:")
    print(df.isnull().sum())
    
    # Ensure key dummy variables are appropriate types (e.g., float or int)
    print("\nData types:")
    print(df.dtypes)
else:
    print("\nSkipping data exploration as the file could not be loaded.")

Successfully loaded data from: Card_Krueger_1994.csv

First 5 rows of the dataframe:
   sheet    fte    nj  after  njafter   dfte
0      1 31.000 1.000  0.000    0.000    NaN
1      1 40.000 1.000  1.000    1.000  9.000
2      2 13.000 1.000  0.000    0.000    NaN
3      2 12.500 1.000  1.000    1.000 -0.500
4      3 12.500 1.000  0.000    0.000    NaN

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sheet    698 non-null    int64  
 1   fte      698 non-null    float64
 2   nj       698 non-null    float64
 3   after    698 non-null    float64
 4   njafter  698 non-null    float64
 5   dfte     349 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 32.8 KB

Descriptive Statistics:
        sheet     fte      nj   after  njafter    dfte
count 698.000 698.000 698.000 698.000  698.000 349.000
mean  245.946  17.784   0.814   0.

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats
from linearmodels.panel import PanelOLS
import os

# --- Configuration ---
# Define the file path (make sure 'Did.csv' is accessible)
file_path = "Did.csv" 

# --- Load Data ---
df = pd.read_csv(file_path)

# Optional: Drop the unnamed index column if it exists from CSV conversion
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# --- Initial Data Exploration ---
print("First 5 rows of the dataframe:")
print(df.head())
print("\nDataFrame Info:")
df.info()
print("\nDescriptive Statistics:")
# Set display precision for better readability
pd.options.display.float_format = '{:.3f}'.format
print(df.describe())
print("\nData types:")
print(df.dtypes)
print(f"\nTotal observations: {len(df)}")
print(f"Number of unique stores (sheets): {df['sheet'].nunique()}")

# Ensure relevant columns are numeric, handling potential non-numeric entries if necessary
cols_to_numeric = ['fte', 'nj', 'after', 'njafter', 'dfte']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce') # 'coerce' turns errors into NaN

# Check for NaNs introduced
print("\nNaN counts after numeric conversion:")
print(df.isnull().sum())
# Consider how to handle NaNs if significant, e.g., df.dropna(subset=cols_to_numeric, inplace=True)
# For this dataset based on the Stata output, we expect few NaNs except in dfte for the 'before' period.

FileNotFoundError: [Errno 2] No such file or directory: 'Did.csv'

### QUESTION 1: Use mean differences to compute the difference in means estimate

**Instructions:**
To calculate the difference-in-differences (DiD) estimate manually, we perform the following steps:
1.  Calculate the average `fte` for each group (NJ and PA) in each time period (before and after).
2.  Calculate the change in average `fte` over time for each group (PA's change and NJ's change).
3.  Calculate the difference between these two changes. This is the DiD estimate.

We will also perform t-tests to compare the means between NJ and PA within each time period.

In [22]:
# --- Manual Calculation of Means and DiD ---

if df is not None:
    # Separate data into before and after periods
    df_before = df[df['after'] == 0].copy()
    df_after = df[df['after'] == 1].copy()

    # Calculate means for each group and time period using pivot_table
    means_pivot = pd.pivot_table(df, values='fte', index='nj', columns='after', aggfunc='mean')
    means_pivot.index = ['PA', 'NJ'] # Rename index (0=PA, 1=NJ)
    means_pivot.columns = ['Before', 'After'] # Rename columns (0=Before, 1=After)

    # Calculate differences over time for each state
    means_pivot['Difference (After - Before)'] = means_pivot['After'] - means_pivot['Before']

    # Calculate differences between states for each period
    diff_before = means_pivot.loc['NJ', 'Before'] - means_pivot.loc['PA', 'Before']
    diff_after = means_pivot.loc['NJ', 'After'] - means_pivot.loc['PA', 'After']

    # Calculate the Difference-in-Differences estimate
    # Difference of the time differences: (NJ_After - NJ_Before) - (PA_After - PA_Before)
    did_estimate = means_pivot.loc['NJ', 'Difference (After - Before)'] - means_pivot.loc['PA', 'Difference (After - Before)']

    # Create a summary table 
    summary_table = pd.DataFrame({
        'Before': [means_pivot.loc['PA', 'Before'], means_pivot.loc['NJ', 'Before'], diff_before],
        'After': [means_pivot.loc['PA', 'After'], means_pivot.loc['NJ', 'After'], diff_after],
        'Difference': [means_pivot.loc['PA', 'Difference (After - Before)'], means_pivot.loc['NJ', 'Difference (After - Before)'], did_estimate]
    }, index=['PA', 'NJ', 'Difference (NJ - PA)'])


    print("--- Mean FTE Employment by State and Period ---")
    print(means_pivot)
    print("\n--- Summary Table (Difference-in-Differences) ---")
    print(summary_table)
    print(f"\nDifference-in-Differences Estimate: {did_estimate:.3f}")

    # --- Optional: T-tests for Mean Comparisons ---
    print("\n--- T-Tests (Comparing NJ vs PA within each period) ---")

    # Before period: NJ (1) vs PA (0)
    pa_before_fte = df_before[df_before['nj'] == 0]['fte']
    nj_before_fte = df_before[df_before['nj'] == 1]['fte']
    # Perform t-test assuming equal variances, like the default 'ttest' command often does
    ttest_before = stats.ttest_ind(nj_before_fte, pa_before_fte, nan_policy='omit', equal_var=True) 
    print(f"T-test Before (NJ vs PA): statistic={ttest_before.statistic:.3f}, pvalue={ttest_before.pvalue:.3f}")
    print(f"  Mean PA Before: {pa_before_fte.mean():.3f}, Mean NJ Before: {nj_before_fte.mean():.3f}, Diff (NJ-PA): {nj_before_fte.mean() - pa_before_fte.mean():.3f}")

    # After period: NJ (1) vs PA (0)
    pa_after_fte = df_after[df_after['nj'] == 0]['fte']
    nj_after_fte = df_after[df_after['nj'] == 1]['fte']
    ttest_after = stats.ttest_ind(nj_after_fte, pa_after_fte, nan_policy='omit', equal_var=True)
    print(f"T-test After (NJ vs PA): statistic={ttest_after.statistic:.3f}, pvalue={ttest_after.pvalue:.3f}")
    print(f"  Mean PA After: {pa_after_fte.mean():.3f}, Mean NJ After: {nj_after_fte.mean():.3f}, Diff (NJ-PA): {nj_after_fte.mean() - pa_after_fte.mean():.3f}")

    print("\nNote: The significance of the DiD estimate itself is typically assessed via regression (see next steps).")
else:
    print("Skipping Question 1 as data was not loaded.")

--- Mean FTE Employment by State and Period ---
    Before  After  Difference (After - Before)
PA  20.300 18.254                       -2.046
NJ  17.301 17.584                        0.283

--- Summary Table (Difference-in-Differences) ---
                      Before  After  Difference
PA                    20.300 18.254      -2.046
NJ                    17.301 17.584       0.283
Difference (NJ - PA)  -2.999 -0.670       2.329

Difference-in-Differences Estimate: 2.329

--- T-Tests (Comparing NJ vs PA within each period) ---
T-test Before (NJ vs PA): statistic=-2.279, pvalue=0.023
  Mean PA Before: 20.300, Mean NJ Before: 17.301, Diff (NJ-PA): -2.999
T-test After (NJ vs PA): statistic=-0.586, pvalue=0.558
  Mean PA After: 18.254, Mean NJ After: 17.584, Diff (NJ-PA): -0.670

Note: The significance of the DiD estimate itself is typically assessed via regression (see next steps).


### Question 2: Estimate the difference-in-difference using a regression model in differences

**Instructions:**
An alternative way to estimate the DiD effect is to use a regression model where the dependent variable is the *change* in the outcome (`dfte`). We regress this change on the treatment indicator (`nj`) and the interaction term (`njafter`).

This regression should be run on a dataset where each row represents one restaurant, using the calculated change `dfte`. This usually means using only the observations from the 'after' period, as `dfte` represents the change leading up to that point. We will estimate this model using Ordinary Least Squares (OLS) first with standard errors, and then with robust standard errors.

In [24]:
# --- Regression in Differences (using dfte) ---

if df is not None:
    # The 'dfte' variable represents the difference (After - Before) for each store.
    # It's typically calculated once per store. Let's use the 'after' period data, 
    # assuming 'dfte' is populated correctly there.
    df_diff_reg_data = df[df['after'] == 1].copy()
    
    # Check for missing 'dfte' values in this subset
    missing_dfte = df_diff_reg_data['dfte'].isnull().sum()
    if missing_dfte > 0:
        print(f"Warning: Found {missing_dfte} missing values in 'dfte' for the 'after' period. Dropping these rows for the regression.")
        df_diff_reg_data.dropna(subset=['dfte'], inplace=True)
        
    print(f"\nNumber of observations for difference regression: {len(df_diff_reg_data)}")

    # --- OLS Regression ---
    # Formula: dfte ~ nj + njafter 
    # The constant represents the change for the control group (PA).
    # The coefficient on 'nj' would represent the baseline difference in *changes* IF nj varied within this subset (it doesn't independently of njafter here).
    # The coefficient on 'njafter' is the DiD estimate (additional change for NJ relative to PA's change).
    # Often, just 'dfte ~ njafter' is sufficient if 'nj' becomes collinear with the constant and 'njafter' in this specific setup. 
    # Let's use a formula that reflects the typical interpretation:
    formula_diff = 'dfte ~ njafter + nj' 
    
    try:
        model_diff = smf.ols(formula_diff, data=df_diff_reg_data)
        results_diff = model_diff.fit()
        print("\n--- Regression in Differences (OLS) ---")
        print(results_diff.summary())
        
        # --- Regression with Robust Standard Errors ---
        # Use HC1 type for standard errors robust to heteroskedasticity
        results_diff_robust = model_diff.fit(cov_type='HC1') 
        print("\n--- Regression in Differences (Robust SE - HC1) ---")
        print(results_diff_robust.summary())

        print("\nInterpretation Hint:")
        print(" - The coefficient for 'njafter' is the DiD estimate.")
        print(" - The constant ('Intercept') represents the average change in 'fte' for the control group (PA).")
        print(" - 'nj' might be dropped or have high standard errors if collinear.")

    except Exception as e:
        print(f"\nAn error occurred during the difference regression: {e}")

else:
    print("Skipping Question 2 as data was not loaded.")


Number of observations for difference regression: 349

--- Regression in Differences (OLS) ---
                            OLS Regression Results                            
Dep. Variable:                   dfte   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.905
Date:                Tue, 01 Apr 2025   Prob (F-statistic):             0.0489
Time:                        01:04:07   Log-Likelihood:                -1244.0
No. Observations:                 349   AIC:                             2492.
Df Residuals:                     347   BIC:                             2500.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

c:\Users\yirsu\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


### Question 3: Estimate the model in levels

**Instructions:**
Now, we estimate the standard DiD regression using the full dataset (both before and after periods). The dependent variable is the level of employment (`fte`). We include dummy variables for the treatment group (`nj`), the post-treatment period (`after`), and their interaction (`njafter`).

The model is: `fte = β₀ + β₁*nj + β₂*after + β₃*njafter + ε`

*   `β₀` is the average `fte` for the control group (PA) before the change.
*   `β₁` is the average difference in `fte` between NJ and PA before the change.
*   `β₂` is the average change in `fte` for the control group (PA) from before to after.
*   `β₃` is the DiD estimate – the additional change in `fte` for the treatment group (NJ) relative to the control group's change.

We will estimate this using OLS with standard errors and then with robust standard errors.

In [25]:
# --- Regression in Levels (using fte) ---

if df is not None:
    # --- OLS Regression ---
    formula_levels = 'fte ~ nj + after + njafter'
    model_levels = smf.ols(formula_levels, data=df)
    results_levels = model_levels.fit()

    print("\n--- Regression in Levels (OLS) ---")
    print(results_levels.summary())

    # --- Regression with Robust Standard Errors ---
    results_levels_robust = model_levels.fit(cov_type='HC1') # Use HC1 robust SEs
    print("\n--- Regression in Levels (Robust SE - HC1) ---")
    print(results_levels_robust.summary())

    print("\nInterpretation Hint:")
    print(" - The coefficient for 'njafter' is the DiD estimate.")
    print(" - 'Intercept': Mean fte for PA (nj=0) Before (after=0).")
    print(" - 'nj': Mean difference NJ - PA Before.")
    print(" - 'after': Mean change After - Before for PA.")

else:
    print("Skipping Question 3 as data was not loaded.")


--- Regression in Levels (OLS) ---
                            OLS Regression Results                            
Dep. Variable:                    fte   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.088
Date:                Tue, 01 Apr 2025   Prob (F-statistic):              0.100
Time:                        01:04:22   Log-Likelihood:                -2519.3
No. Observations:                 698   AIC:                             5047.
Df Residuals:                     694   BIC:                             5065.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20

### QUESTION 4: Estimate the levels model with clustered standard errors

**Instructions:**
Observations within the same restaurant (`sheet`) might be correlated over time (e.g., a restaurant that started with high employment might tend to stay high). Standard OLS assumes independent observations, which might not hold here. To account for this potential within-restaurant correlation, we re-estimate the levels model from Question 3, but this time we cluster the standard errors by `sheet`.

Compare the standard errors obtained here (especially for the `njafter` coefficient) with those from the OLS and robust estimations in Question 3. Does clustering increase or decrease the standard errors in this case?

In [26]:
# --- Regression in Levels with Clustered Standard Errors ---

if df is not None and results_levels is not None and results_levels_robust is not None:
    # Ensure the clustering variable 'sheet' doesn't have missing values 
    # in the data used by the model (OLS uses listwise deletion by default)
    valid_indices = results_levels.model.data.row_labels # Get indices used in the fitted model
    cluster_groups = df.loc[valid_indices, 'sheet']

    # Estimate the model with standard errors clustered by 'sheet'
    results_levels_clustered = model_levels.fit(cov_type='cluster', 
                                                cov_kwds={'groups': cluster_groups})

    print("\n--- Regression in Levels (Clustered SE by sheet) ---")
    print(results_levels_clustered.summary())

    # --- Comparison of Standard Errors ---
    print("\n--- Standard Error Comparison for 'njafter' Coefficient ---")
    try:
        ols_se = results_levels.bse['njafter']
        robust_se = results_levels_robust.bse['njafter']
        clustered_se = results_levels_clustered.bse['njafter']
        
        print(f"OLS Standard Error:          {ols_se:.4f}")
        print(f"Robust SE (HC1):           {robust_se:.4f}")
        print(f"Clustered SE (by sheet):   {clustered_se:.4f}")

        if clustered_se < ols_se and clustered_se < robust_se:
             print("\nObservation: Clustering standard errors decreased the SE for 'njafter' compared to OLS and standard Robust SE.")
        elif clustered_se > ols_se or clustered_se > robust_se:
             print("\nObservation: Clustering standard errors increased the SE for 'njafter' compared to OLS and/or standard Robust SE.")
        else:
             print("\nObservation: Clustering standard errors resulted in a similar SE for 'njafter' compared to OLS / Robust SE.")
             
    except KeyError:
        print("\nCould not retrieve standard errors for comparison (perhaps model estimation failed).")
    except NameError:
        print("\nCould not retrieve standard errors for comparison (previous model results not available).")
        
else:
    print("Skipping Question 4 as data or previous results were not available.")


--- Regression in Levels (Clustered SE by sheet) ---
                            OLS Regression Results                            
Dep. Variable:                    fte   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.223
Date:                Tue, 01 Apr 2025   Prob (F-statistic):              0.301
Time:                        01:04:56   Log-Likelihood:                -2519.3
No. Observations:                 698   AIC:                             5047.
Df Residuals:                     694   BIC:                             5065.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

### QUESTION 5: Estimate the levels model using fixed effects

**Instructions:**
Fixed effects regression controls for all time-invariant characteristics of each restaurant (e.g., location, management quality, initial size category). We use `sheet` as the entity identifier for the fixed effects. This is often implemented using the "within" estimator.

Estimate the model `fte ~ nj + after + njafter` including restaurant fixed effects. Observe which variables are estimated and which, if any, are dropped. Explain *why* any variables are dropped. We will use the `PanelOLS` estimator from the `linearmodels` library.

In [33]:
# --- Fixed Effects (Within) Regression using PanelOLS ---

if df is not None:
    try:
        # Prepare data for PanelOLS: Set BOTH entity ('sheet') and time ('after') as the index.
        df_panel = df.copy()
        
        # Define key columns for checking NaNs and potential dropping
        key_cols = ['sheet', 'after', 'fte', 'nj', 'njafter']
        initial_rows = len(df_panel)
        
        # Check for and handle missing values in key columns before setting index
        if df_panel[key_cols].isnull().any().any():
             print(f"Warning: Data contains NaNs in key columns {key_cols}. Dropping rows with NaNs in these columns.")
             df_panel.dropna(subset=key_cols, inplace=True)
             print(f"Dropped {initial_rows - len(df_panel)} rows due to NaNs.")
        
        # Ensure 'sheet' and 'after' are suitable types for index (e.g., int or category)
        df_panel['sheet'] = df_panel['sheet'].astype(int)
        df_panel['after'] = df_panel['after'].astype(int)

        # Set the MultiIndex
        df_panel = df_panel.set_index(['sheet', 'after'])

        # *** Workaround: Create a duplicate 'after' column for use in the formula ***
        # This column is needed because set_index removes 'after' from the regular columns.
        df_panel['after_col'] = df_panel.index.get_level_values('after')

        # Define and estimate the model including entity (restaurant) fixed effects
        # Formula uses 'after_col' instead of 'after'. 
        # 'nj' is included to explicitly see it get dropped.
        # Add 'EntityEffects' to the formula.
        formula_fe = 'fte ~ njafter + after_col + nj + EntityEffects' 
        
        print("\nAttempting PanelOLS estimation...")
        mod_fe = PanelOLS.from_formula(formula_fe, data=df_panel, drop_absorbed=True)
        results_fe = mod_fe.fit()

        print("\n--- Fixed Effects (PanelOLS) Regression ---")
        print(results_fe)
        
        # Explicitly check which variables were dropped (absorbed)
        print("\nVariables included in final FE model:", results_fe.params.index.tolist())
        print("Note: 'nj' was likely dropped (absorbed by EntityEffects). The constant is also absorbed.")


    except pd.errors.DuplicateLabelError as e:
         print(f"\nError: Duplicate index entries found. Check if any sheet/after combination appears more than once. {e}")
         results_fe = None
    except KeyError as e:
        print(f"\nAn error occurred during Fixed Effects estimation: A required column is missing - {e}")
        results_fe = None
    except Exception as e:
        print(f"\nAn unexpected error occurred during the Fixed Effects estimation: {e}")
        print("This might happen if the data structure is not correctly recognized as a panel.")
        results_fe = None 

else:
    print("Skipping Question 5 as data was not loaded.")


Attempting PanelOLS estimation...

--- Fixed Effects (PanelOLS) Regression ---
                          PanelOLS Estimation Summary                           
Dep. Variable:                    fte   R-squared:                        0.0114
Estimator:                   PanelOLS   R-squared (Between):             -0.0094
No. Observations:                 698   R-squared (Within):               0.0114
Date:                Tue, Apr 01 2025   R-squared (Overall):             -0.0084
Time:                        01:10:10   Log-likelihood                   -2004.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.0070
Entities:                         349   P-value                           0.1359
Avg Obs:                       2.0000   Distribution:                   F(2,347)
Min Obs:                       2.0000                                           
Max Obs:                     

C:\Users\yirsu\AppData\Local\Temp\ipykernel_39624\3729254006.py:37: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

nj

  results_fe = mod_fe.fit()


--- Explanation of Dropped Variables (Fixed Effects) ---

**Variable(s) Dropped:**
1.  `nj`: The variable indicating whether a restaurant is in New Jersey.
2.  The overall constant (Intercept).

**Reason:**
*   **`nj`:** As explained before, the fixed effects estimator controls for all time-invariant characteristics specific to each restaurant (`sheet`). Since a restaurant's location (`nj`) doesn't change over time, its effect is perfectly collinear with the restaurant's unique fixed effect. It cannot be estimated separately and is absorbed/dropped.
*   **Constant:** The entity fixed effects (one for each restaurant, minus a reference category) act like a set of dummy variables. This set of dummies is perfectly collinear with an overall constant term. Therefore, the constant is typically dropped when entity fixed effects are included to avoid multicollinearity (the dummy variable trap). The baseline level for each entity is captured by its specific fixed effect.